In [ ]:
import sys; sys.path.append('..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer

l = elastic_rods.RodLinkage('../../examples/actuation_sparsification/half_vase_opened.msh')
mat = elastic_rods.RodMaterial('+', 40000, 0.3, [2.0, 2.0, 0.2, 0.2], stiffAxis=elastic_rods.StiffAxis.D1)
l.setMaterial(mat)

jdo = l.dofOffsetForJoint(l.centralJoint())
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
with suppress_stdout(): elastic_rods.compute_equilibrium(l, l.averageJointAngle, fixedVars=fixedVars)
view = LinkageViewer(l, width=1024)
view.setCameraParams(((0.4075185454716348, 3.1892642201046715, 0.3099480992441177),
 (0.15364528336486324, 0.2839547329660347, 0.9464474821805594),
 (0.0, 0.0, 0.0)))
view.show()

In [ ]:
l.updateRotationParametrizations()

In [ ]:
l.saveVisualizationGeometry('test.msh')

In [ ]:
dofs = l.getDoFs()

In [ ]:
# dofs[l.dofOffsetForJoint(0) + 5] += np.pi / 8
# dofs[l.dofOffsetForJoint(315) + 5] += np.pi / 8
dofs[jdo + 5] += np.pi / 8
l.setDoFs(dofs)
view.update()

In [ ]:
# Output a file with the maximum stretching/bending/twisting stresses
# visualized on the beams. This file can be viewed in GMsh. Note that
# the entire cross-section is colored by the maximum stress appearing
# anywhere in that cross-section.
l.saveStressVisualization('stressVis.msh')

In [ ]:
# Output a file visualizing the forces and torques acting on each joint.
# We show the forces of the "A" rods acting on the joint (which are countered
# by equal and opposte forces from the "B" rods.
l.updateRotationParametrizations() # Needed to ensure forces/torques are output
l.writeLinkageDebugData('jointForces.msh')

In [ ]:
# Get the stress measures as scalar fields on the centerline for each rod of the linkage.
# These are lists of numpy arrays (one array per rod) where each array holds the value of the
# scalar field along the rod's centerline. In the case of bending stress, the array
# actually holds two scalar fields, one in each column.

# Get the z-stress due to axial stretching
stretchingStresses = np.array([s.rod.stretchingStresses() for s in l.segments()])
# Get the (min, max) bending z-stress over the cross-section.
bendingStresses = np.array([s.rod.bendingStresses() for s in l.segments()])
# Get the principal stresses due to the shearing caused by rod torsion.
twistingStresses = np.array([s.rod.twistingStresses() for s in l.segments()])

In [ ]:
np.min(bendingStresses), np.max(bendingStresses)

In [ ]:
np.max(twistingStresses)

In [ ]:
np.min(stretchingStresses), np.max(stretchingStresses)

In [ ]:
l.updateRotationParametrizations()
# Calculate the net force and torque exerted on each joint of the linkage by the "A" rods.
# Note: forces are in N, torques are in N mm!
rf = l.rivetNetForceAndTorques()
jointForces  = rf[:, 0:3]
jointTorques = rf[:, 3:6]

def reportGreatestMagnitude(name, data, unit):
    mags = np.linalg.norm(data, axis=1)
    worstJoint = np.argmax(mags)
    print(f'Greatest {name} on joint {worstJoint}: {data[worstJoint, :]}\n\t\t(magnitude {np.max(mags)} {unit})')

reportGreatestMagnitude('force', rf[:, 0:3], 'N')
reportGreatestMagnitude('torque', rf[:, 3:6], 'N mm')

In [ ]:
shearForces = []
shearTorques = []
for ji in range(l.numJoints()):
    n = l.joint(ji).normal
    F = jointForces[ji, :]
    T = jointTorques[ji, :]
    shearForces.append(np.linalg.norm(F - np.dot(F, n) * n))
    shearTorques.append(np.linalg.norm(T - np.dot(T, n) * n ))
shearForces = np.array(shearForces)
shearTorques = np.array(shearTorques)

In [ ]:
j = l.joint(18)

In [ ]:
np.max(shearForces)

In [ ]:
np.argmax(shearTorques), np.max(shearTorques)

In [ ]:
np.dot(jointTorques[218, :], l.joint(218).normal)

In [ ]:
l.gradient()[l.dofOffsetForJoint(218) + 6]

In [ ]:
l.rivetForces()[l.dofOffsetForJoint(218) + 6] * 2

## Boundary conditions at material interface

In [ ]:
rodWidth = 300
npts = 200
midpt = (npts + 1) // 2
thetaOffset = 3 * npts

In [ ]:
pts = np.pad(np.linspace(-rodWidth / 2, rodWidth / 2, npts)[:,np.newaxis], [(0, 0), (0, 2)], mode='constant')

In [ ]:
r = elastic_rods.ElasticRod(pts)

In [ ]:
r.setMaterial(elastic_rods.RodMaterial('rectangle', 4.0e10 / 1e6, 0.3, [12, 8]))

In [ ]:
import linkage_vis
view2 = linkage_vis.LinkageViewer(r)
view2.show()

In [ ]:
dofs = r.getDoFs()
dofs[0:3] = [-120, -10, 0]
dofs[(npts - 1) * 3:npts * 3] = [120, -10, 0]

In [ ]:
r.setDoFs(dofs)

In [ ]:
to = r.thetaOffset()
fixedVars = [0, 1, 2, (npts - 1) * 3, (npts - 1) * 3 + 1, (npts - 1) * 3 + 2, to]

In [ ]:
with suppress_stdout(): elastic_rods.compute_equilibrium(r, fixedVars=fixedVars)
# Now fix the slope too to prevent rotation of the reference frame
fixedVars += [3, 4, 5, (npts - 2) * 3, (npts - 2) * 3 + 1, (npts - 2) * 3 + 2]

In [ ]:
# Apply a twist to the rod as well...
fixedVars += [r.numDoF() - 1]

In [ ]:
dofs = r.getDoFs()
dofs[to] = -1.5 * np.pi
dofs[r.numDoF() - 1] = 1.5 * np.pi
r.setDoFs(dofs)

In [ ]:
with suppress_stdout(): elastic_rods.compute_equilibrium(r, fixedVars=fixedVars)

In [ ]:
view2.update()

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(8,4))
x = np.array(r.deformedPoints())[:, 0]
plt.plot(x, r.bendingStresses()[:, 0], label='bending stress', linewidth=5)
plt.plot(x, r.twistingStresses(), label='twisting stress', linewidth=5)
plt.legend()
plt.show()

In [ ]:
gsm = elastic_rods.GradientStencilMaskCustom()

nv = r.numVertices()
ne = r.numEdges()
mask = np.zeros(nv, dtype=bool)
vtxMin, vtxMax = (nv // 2 - 5, nv // 2 + 5)
edgeMin, edgeMax = (vtxMin - 1, vtxMax)
mask[vtxMin:vtxMax] = True
gsm.vtxStencilMask = mask
mask = np.zeros(ne, dtype=bool)
mask[edgeMin:edgeMax] = True
gsm.edgeStencilMask = mask

In [ ]:
et = elastic_rods.EnergyType.Full

In [ ]:
r.gradient(energyType=et, stencilMask=gsm)[3 * (vtxMin - 2):3 * (vtxMax + 2)]

In [ ]:
r.gradient(energyType=et, stencilMask=gsm)[to + edgeMin - 1:to + edgeMax + 1]

In [ ]:
g = r.gradient(stencilMask=gsm)

## Forces acting at a joint

In [ ]:
j = l.joint(1)
j.segments_A, j.segments_B

In [ ]:
j.terminalEdgeIdentification(542)

In [ ]:
j.isStartB # rod b is segment 183 and 465 pieced together in un-reversed orientation

In [ ]:
r = l.segment(j.segments_B[1]).rod
r2 = l.segment(j.segments_B[0]).rod

In [ ]:
jointToVtxForces = r.gradient()[0:6] + r2.gradient()[3 * (r2.numVertices() - 2):3 * r2.numVertices()]
aroundEdgeTorque = r.gradient()[r.thetaOffset()] + r2.gradient()[r.numDoF() - 1]

In [ ]:
l.rivetNetForceAndTorques()[1, :]

In [ ]:
jointToVtxForces[0:3] + jointToVtxForces[3:6]

In [ ]:
dc = r.deformedConfiguration()
t = dc.tangent[0]
e = dc.len[0] * t
N = j.normal
dc.len[0] * dc.tangent[0]
tperp = np.cross(N, t)
torqueOnJoint = l.rivetNetForceAndTorques()[1, 3:6]

In [ ]:
jointToVtxTorque = np.cross(-e / 2, jointToVtxForces[0:3]) + np.cross(e / 2, jointToVtxForces[3:6])

In [ ]:
[np.dot(torqueOnJoint, v) for v in [N, t, tperp]]

In [ ]:
[np.dot(jointToVtxTorque, N), aroundEdgeTorque, np.dot(jointToVtxTorque, tperp)]

In [ ]:
(-3.59545902e-02 + 3.42758665e-02, 1.47690205e-01 + -1.48400602e-01,
-3.59545902e-02 + 3.42758665e-02 + (1.47690205e-01 + -1.48400602e-01))

In [ ]:
nkeptVtx = 5;
gsm_r1 = elastic_rods.GradientStencilMaskCustom()
gsm_r2 = elastic_rods.GradientStencilMaskCustom()
mask = np.zeros(r.numVertices(), dtype=bool)
mask[0:nkeptVtx] = True
gsm_r1.vtxStencilMask = mask
mask[0:nkeptVtx] = False
mask[-nkeptVtx:] = True
gsm_r2.vtxStencilMask = mask
mask = np.zeros(r.numEdges(), dtype=bool)
mask[0:nkeptVtx] = True
gsm_r1.edgeStencilMask = mask
mask[0:nkeptVtx] = False
mask[-nkeptVtx:] = True
gsm_r2.edgeStencilMask = mask

r.gradient(stencilMask=gsm_r1)

In [ ]:
r2.gradient(stencilMask=gsm_r2)

In [ ]:
partialGrad1 = r.gradient(stencilMask=gsm_r1)
partialGrad2 = r2.gradient(stencilMask=gsm_r2)

In [ ]:
np.mean(r.deformedPoints()[0:2], axis=0)

In [ ]:
r.deformedPoints()[0:2]

In [ ]:
# Get the net force and torque on an edge along with the application point
def netForceAndTorqueOnEdge(rod, g, edgeIdx):
    F1 = g[3 * edgeIdx:3 * (edgeIdx + 1)]
    F2 = g[3 * (edgeIdx + 1):3 * (edgeIdx + 2)]
    dc = rod.deformedConfiguration()
    edgeMidpt = np.mean(rod.deformedPoints()[edgeIdx:edgeIdx+2], axis=0)
    t = dc.tangent[edgeIdx]
    e = t * dc.len[edgeIdx]
    torqueAround = g[rod.thetaOffset() + edgeIdx]
    return (edgeMidpt, np.array([F1 + F2, np.cross(-e / 2, F1) + np.cross(e / 2, F2) + torqueAround * t]))

In [ ]:
internalInterfaceLoads1 = netForceAndTorqueOnEdge(r, partialGrad1, nkeptVtx - 1)
internalInterfaceLoads2 = netForceAndTorqueOnEdge(r2, partialGrad2, r2.numEdges() - nkeptVtx)

In [ ]:
totalForceAndTorqueOnJointEdge = (netForceAndTorqueOnEdge(r, r.gradient(), 0)[1] +
                                  netForceAndTorqueOnEdge(r2, r2.gradient(), r2.numEdges() - 1)[1])

In [ ]:
[np.dot(torqueOnJoint, v) for v in [N, t, tperp]]

In [ ]:
[np.dot(totalForceAndTorqueOnJointEdge[1,:], v) for v in [N, t, tperp]]

In [ ]:
totalForceAndTorqueOnJointEdge

In [ ]:
netInterfaceForce = internalInterfaceLoads1[1][0] + internalInterfaceLoads2[1][0]

In [ ]:
# Compute the torque around the joint position applied by
# a net force and torque at point load[0].
def torqueAroundJointPos(load):
    pt = load[0]
    return load[1][1] + np.cross(pt - j.position, load[1][0])

In [ ]:
torqueAroundJointPos(internalInterfaceLoads1) + torqueAroundJointPos(internalInterfaceLoads2)

In [ ]:
print(torqueAroundJointPos(internalInterfaceLoads1))
print(torqueAroundJointPos(internalInterfaceLoads2))

## Extract "free body diagram" for piece of rod around joint

In [ ]:
import importlib, structural_analysis
importlib.reload(structural_analysis)
from structural_analysis import Load, isolateRodPieceAtJoint, getLoadOnEdge, freeBodyDiagramReport

isolateRodPieceAtJoint(l, 1, 1, keepEdges=5)

In [ ]:
freeBodyDiagramReport(l, 1, 1, keepEdges=5)